In [1]:
import paramiko
import time
import sys

ip = "172.16.16.25"
user = "root"
pw = "asd123!@"

# 고정된 경로
HTTPD_CONF = ""
VHOST_DIR = ""

# 동적으로 받을 값
WEB_DOCUMENT_ROOT = ""
VHOST_CONF = ""
SERVER_NAME = ""
ERROR_LOG = ""
CUSTOM_LOG = ""


def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def set_firewalld():
    cc("systemctl stop firewalld; systemctl disable firewalld")

def install_httpd():
    cc("dnf -y install httpd ; systemctl enable --now httpd")

def set_user_dir_and_vhost_conf_rocky():
    global WEB_DOCUMENT_ROOT
    WEB_DOCUMENT_ROOT = input("사용할 웹 디렉토리를 입력하세요 (예: /home/team1 또는 /var/www/young): ").strip()
    cc(f"mkdir -p {WEB_DOCUMENT_ROOT} && chmod 755 {WEB_DOCUMENT_ROOT}")
    print(f"✅ 웹 문서 루트: {WEB_DOCUMENT_ROOT}")

def set_vhostconf_rocky():
  #  VirtualHost 설정 -> 가상 호스트 파일이 있으면 우선 적용이기 떄문에 
  # /etc/httpd/conf/httpd.conf의 124, 129,136줄을 수정하지 않아도 됩니다.
    
    global VHOST_CONF, SERVER_NAME, ERROR_LOG, CUSTOM_LOG
    vhost_name = input("VHost 설정 파일을 만들겠습니다 원하는 이름을 입력하세요 (예: team1.conf): ")
    
    SERVER_NAME = input("ServerName을 입력하세요 (예: test.local): ")
    ERROR_LOG = input("ErrorLog 경로를 입력하세요 (예: /var/log/httpd/team1_error.log): ")
    CUSTOM_LOG = input("CustomLog 경로를 입력하세요 (예: /var/log/httpd/team1_access.log): ")

    
    VHOST_CONF = f"/etc/httpd/conf.d/{vhost_name}"
    print(f" {VHOST_CONF} 생성 중...")
    cmd = f"""
cat <<EOF > "{VHOST_CONF}"
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName {SERVER_NAME}
    DocumentRoot {WEB_DOCUMENT_ROOT}
    
    <Directory {WEB_DOCUMENT_ROOT}>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

    ErrorLog {ERROR_LOG}
    CustomLog {CUSTOM_LOG} combined
</VirtualHost>
EOF
"""
    cc(cmd)

def mk_index():
    """기본 index.html 생성"""
    print(" index.html 생성 중...")
    cmd = f"echo '<h1>This is index.html for Test by mk_index func</h1>' > {WEB_DOCUMENT_ROOT}/index.html"
    print(f"{WEB_DOCUMENT_ROOT} 경로에 파일이 생성되었습니다")
    cc(cmd)

def restart_httpd():
    cc("systemctl restart httpd")
    print("Apache 재시작 완료!")

def install_vsftpd():
    print("vsftpd 설치 중...")
    cc("dnf -y install vsftpd")
    cc("systemctl enable --now vsftpd")
    print("systemctl status vsftpd")

def go():
    set_firewalld()
    install_httpd()
    set_user_dir_and_vhost_conf_rocky()
    set_vhostconf_rocky()
    mk_index()
    restart_httpd()
    install_vsftpd()

go()
####################################################################
def lamp_ftp() : ## 완료
    return """
    #!/bin/bash
set -e

echo "===== Rocky Linux 서버 초기화 스크립트 시작 ====="

# 0️⃣ firewalld 비활성화
echo "firewalld 비활성화 중..."
systemctl stop firewalld
systemctl disable firewalld
echo "firewalld 비활성화 완료!"


# 3️⃣ Apache 설치
echo "Apache 설치 중..."
dnf -y install httpd
systemctl enable --now httpd
echo "Apache 설치 및 자동 시작 완료!"

# 4️⃣ 사용자 웹 디렉토리 생성
WEB_DIR="/home/heeyoung"
mkdir -p "$WEB_DIR"
chmod 755 "$WEB_DIR"
chown -R $USER:$USER "$WEB_DIR"

# SELinux 활성화 시 컨텍스트 적용
if sestatus | grep -q "enabled"; then
    chcon -R -t httpd_sys_content_t "$WEB_DIR"
fi
echo "웹 디렉토리 생성 및 권한 설정 완료!"

# 5️⃣ VirtualHost 설정
VHOST_CONF="/etc/httpd/conf.d/heeyoung.conf"
cat <<EOF > "$VHOST_CONF"
<VirtualHost *:80>
    ServerName www.heeyoung.com
    DocumentRoot "$WEB_DIR"
    <Directory "$WEB_DIR">
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>
</VirtualHost>
EOF
echo "VirtualHost 설정 완료: $VHOST_CONF"

# 6️⃣ index.html 생성
echo "qqq" > "$WEB_DIR/index.html"
echo "index.html 생성 완료!"

# 7️⃣ Apache 재시작
systemctl restart httpd
echo "Apache 재시작 완료!"

# 8️⃣ vsftpd 설치 및 시작
echo "vsftpd 설치 중..."
dnf -y install vsftpd
systemctl enable --now vsftpd
echo "vsftpd 설치 및 자동 시작 완료!"

echo "🎉 모든 설정 완료! 웹 서버와 FTP 서버가 준비되었습니다."

    """


#bash_script = lamp_ftp()

#run_remote(bash_script)


Last metadata expiration check: 3:59:08 ago on Tue Nov  4 08:53:42 2025.
Package httpd-2.4.62-4.el9_6.4.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!



사용할 웹 디렉토리를 입력하세요 (예: /home/team1 또는 /var/www/young):  /home/a



✅ 웹 문서 루트: /home/a


VHost 설정 파일을 만들겠습니다 원하는 이름을 입력하세요 (예: team1.conf):  a.conf
ServerName을 입력하세요 (예: test.local):  a.local
ErrorLog 경로를 입력하세요 (예: /var/log/httpd/team1_error.log):  /var/log/httpd/a.log
CustomLog 경로를 입력하세요 (예: /var/log/httpd/team1_access.log):  /var/log/httpd/ae.log


 /etc/httpd/conf.d/a.conf 생성 중...

 index.html 생성 중...
/home/a 경로에 파일이 생성되었습니다


Apache 재시작 완료!
vsftpd 설치 중...
Last metadata expiration check: 4:00:06 ago on Tue Nov  4 08:53:42 2025.
Package vsftpd-3.0.5-6.el9.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!


systemctl status vsftpd
